In [1]:
import pandas as pd
import numpy as np
import re
from datetime import datetime, timedelta

In [2]:
stats_equipos = pd.read_csv("./input.csv")
distancias = pd.read_csv("./distancias.csv")
stats_jugadores = pd.read_csv ("./Stats_jugadores_27_04_20.csv")
stats_equipos = stats_equipos.drop_duplicates(['ID Partido'], keep='last')

C:\Users\34680\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
stats_jugadores_ordered = pd.DataFrame(columns=('ID Partido', 'visitor_prorrogue', 'local_prorrogue'))
stats_jugadores = stats_jugadores.dropna()
stats_jugadores_ordered ['ID Partido'] = stats_jugadores ['ID Partido'].unique()
                                       

In [4]:
stats_jugadores['Minutes Played'] = stats_jugadores['Minutes Played'].replace('\:', '.', regex = True)
stats_jugadores['Minutes Played'] = pd.to_numeric(stats_jugadores['Minutes Played'])

In [5]:
stats_jugadores = stats_jugadores.groupby(['ID Partido', 'Team', 'Local']).sum().reset_index()

In [6]:
stats_jugadores_ordered

,ID Partido,visitor_prorrogue,local_prorrogue
0,/boxscores/201510280BOS.html,NaN,NaN
1,/boxscores/201510300PHI.html,NaN,NaN
2,/boxscores/201511020PHI.html,NaN,NaN
3,/boxscores/201511040MIL.html,NaN,NaN
4,/boxscores/201511060CLE.html,NaN,NaN
...,...,...,...
6215,/boxscores/201912060CHO.html,NaN,NaN
6216,/boxscores/201912110BRK.html,NaN,NaN
6217,/boxscores/201912170NOP.html,NaN,NaN
6218,/boxscores/202002220CHO.html,NaN,NaN


In [7]:
for i in stats_jugadores_ordered.index:
    borrador = stats_jugadores[stats_jugadores_ordered.loc[i, 'ID Partido'] == stats_jugadores['ID Partido']]
    if (borrador.loc[borrador.index.min(), 'Minutes Played'] > 50.0):
        stats_jugadores_ordered.loc[i, 'visitor_prorrogue'] = 1
        stats_jugadores_ordered.loc[i, 'local_prorrogue'] = 1
    else:
        stats_jugadores_ordered.loc[i, 'visitor_prorrogue'] = 0
        stats_jugadores_ordered.loc[i, 'local_prorrogue'] = 0
                

In [8]:
stats_equipos = pd.merge(stats_equipos, stats_jugadores_ordered, on=['ID Partido'], how='left')

In [9]:
stats_equipos['number_date'] = stats_equipos['ID Partido'].replace('[A-z]', '', regex = True)
stats_equipos['number_date'] = stats_equipos['number_date'].replace('\.', '', regex = True)
stats_equipos['number_date'] = stats_equipos['number_date'].replace('\/', '', regex = True)
stats_equipos['number_date'] = stats_equipos['number_date'].str.slice(0,8)

In [10]:
stats_equipos['number_date'] = pd.to_datetime(stats_equipos['number_date'], format='%Y%m%d', errors='ignore')

In [11]:
stats_equipos['previous_date'] = stats_equipos['number_date'] - timedelta(days =1)
stats_equipos['previous_date'] = pd.to_datetime(stats_equipos['previous_date'], format='%Y%m%d', errors='ignore')
stats_equipos['two_days_ago'] = stats_equipos['number_date'] - timedelta(days =2)
stats_equipos['two_days_ago'] = pd.to_datetime(stats_equipos['two_days_ago'], format='%Y%m%d', errors='ignore')
stats_equipos['three_days_ago'] = stats_equipos['number_date'] - timedelta(days =1)
stats_equipos['three_days_ago'] = pd.to_datetime(stats_equipos['three_days_ago'], format='%Y%m%d', errors='ignore')


In [12]:
stats_equipos['local_dif_between_previous_game'] = 0
stats_equipos['visitor_dif_between_previous_game'] = 0
stats_equipos['local_dif_between_previous_game'] = pd.to_datetime(stats_equipos['local_dif_between_previous_game'], format='%Y%m%d', errors='ignore')
stats_equipos['visitor_dif_between_previous_game'] = pd.to_datetime(stats_equipos['visitor_dif_between_previous_game'], format='%Y%m%d', errors='ignore')

In [13]:
stats_equipos['local_played_previous_date'] = 0
stats_equipos['visitor_played_previous_date'] = 0
stats_equipos['local_played_two_days_ago'] = 0
stats_equipos['visitor_played_two_days_ago'] = 0
stats_equipos['local_played_three_days_ago'] = 0
stats_equipos['visitor_played_three_days_ago'] = 0


stats_equipos['local_played_prorrogue_previous_date'] = 0
stats_equipos['visitor_played_prorrogue_previous_date'] = 0
stats_equipos['local_played_prorrogue_two_days_ago'] = 0
stats_equipos['visitor_played_prorrogue_two_days_ago'] = 0
stats_equipos['local_played_prorrogue_three_days_ago'] = 0
stats_equipos['visitor_played_prorrogue_three_days_ago'] = 0


stats_equipos['previous_10_days'] = stats_equipos['number_date'] - timedelta(days =10)
stats_equipos['previous_10_days'] = pd.to_datetime(stats_equipos['previous_10_days'], format='%Y%m%d', errors='ignore')
stats_equipos['local_played_local'] = 0
stats_equipos['local_played_visitor'] = 0
stats_equipos['visitor_played_visitor'] = 0
stats_equipos['visitor_played_local'] = 0
stats_equipos ['place_local_played'] = 'Nada'
stats_equipos ['place_visitor_played'] = 'Nada'
stats_equipos ['avg_distance_place_visitor_played'] = -1
stats_equipos ['avg_distance_place_local_played'] = -1



for actual in stats_equipos.index:
    
    jugo_dia_anterior = stats_equipos[(stats_equipos ['number_date'] == stats_equipos.loc[actual,'previous_date'])]
    jugo_dos_dias_antes = stats_equipos[(stats_equipos ['number_date'] == stats_equipos.loc[actual,'two_days_ago'])]
    jugo_tres_dias_antes = stats_equipos[(stats_equipos ['number_date'] == stats_equipos.loc[actual,'three_days_ago'])]
    
    visitor_jugo_dia_anterior = jugo_dia_anterior[((stats_equipos['local_team'] == stats_equipos.loc[actual,'visitor_team'])|(stats_equipos['visitor_team'] == stats_equipos.loc[actual,'visitor_team']))]
    local_jugo_dia_anterior = jugo_dia_anterior[((stats_equipos['local_team'] == stats_equipos.loc[actual,'local_team'])|(stats_equipos['visitor_team'] == stats_equipos.loc[actual,'local_team']))]
    
    visitor_jugo_dos_dias_antes = jugo_dos_dias_antes[((stats_equipos['local_team'] == stats_equipos.loc[actual,'visitor_team'])|(stats_equipos['visitor_team'] == stats_equipos.loc[actual,'visitor_team']))]
    local_jugo_dos_dias_antes = jugo_dos_dias_antes[((stats_equipos['local_team'] == stats_equipos.loc[actual,'local_team'])|(stats_equipos['visitor_team'] == stats_equipos.loc[actual,'local_team']))]
    
    visitor_jugo_tres_dias_antes = jugo_tres_dias_antes[((stats_equipos['local_team'] == stats_equipos.loc[actual,'visitor_team'])|(stats_equipos['visitor_team'] == stats_equipos.loc[actual,'visitor_team']))]
    local_jugo_tres_dias_antes = jugo_tres_dias_antes[((stats_equipos['local_team'] == stats_equipos.loc[actual,'local_team'])|(stats_equipos['visitor_team'] == stats_equipos.loc[actual,'local_team']))]
    
    diez_dias_previos = stats_equipos[(stats_equipos['number_date'] > stats_equipos.loc[actual,'previous_10_days']) & (stats_equipos['number_date'] < (stats_equipos.loc[actual,'number_date']))]
    
    if not(local_jugo_dia_anterior.empty):
        stats_equipos.loc[actual,'local_played_previous_date'] = 1
        if (local_jugo_dia_anterior.loc[local_jugo_dia_anterior.index.min(),'local_prorrogue'] == 1):
            stats_equipos.loc[actual,'local_played_prorrogue_previous_date'] = 1
   
    if not (visitor_jugo_dia_anterior.empty):
        stats_equipos.loc[actual,'visitor_played_previous_date'] = 1
        if (visitor_jugo_dia_anterior.loc[visitor_jugo_dia_anterior.index.min(),'visitor_prorrogue'] == 1):
            stats_equipos.loc[actual,'visitor_played_prorrogue_previous_date'] = 1
    
    if not(local_jugo_dos_dias_antes.empty):
        stats_equipos.loc[actual,'local_played_two_days_ago'] = 1
        if (local_jugo_dos_dias_antes.loc[local_jugo_dos_dias_antes.index.min(),'local_prorrogue'] == 1):
            stats_equipos.loc[actual,'local_played_prorrogue_two_days_ago'] = 1
    
    if not(visitor_jugo_dos_dias_antes.empty):
        stats_equipos.loc[actual,'visitor_played_two_days_ago'] = 1
        if (visitor_jugo_dos_dias_antes.loc[visitor_jugo_dos_dias_antes.index.min(),'visitor_prorrogue'] == 1):
            stats_equipos.loc[actual,'visitor_played_prorrogue_two_days_ago'] = 1    
    
    if not(local_jugo_tres_dias_antes.empty):
        stats_equipos.loc[actual,'local_played_three_days_ago'] = 1
        if (local_jugo_tres_dias_antes.loc[local_jugo_tres_dias_antes.index.min(),'local_prorrogue'] == 1):
            stats_equipos.loc[actual,'local_played_prorrogue_three_days_ago'] = 1
    
    if not(visitor_jugo_tres_dias_antes.empty):
        stats_equipos.loc[actual,'visitor_played_three_days_ago'] = 1
        if (visitor_jugo_tres_dias_antes.loc[visitor_jugo_tres_dias_antes.index.min(),'visitor_prorrogue'] == 1):
            stats_equipos.loc[actual,'visitor_played_prorrogue_three_days_ago'] = 1    
    
    
    if not (diez_dias_previos.empty):                                 

        local_jugo_local = diez_dias_previos[((diez_dias_previos['local_team'] == stats_equipos.loc[actual,'local_team']))]        
        local_jugo_visitante = diez_dias_previos[((diez_dias_previos['visitor_team'] == stats_equipos.loc[actual,'local_team']))]   
        visitante_jugo_local = diez_dias_previos[((diez_dias_previos['local_team'] == stats_equipos.loc[actual,'visitor_team']))]
        visitante_jugo_visitante = diez_dias_previos[((diez_dias_previos['visitor_team'] == stats_equipos.loc[actual,'visitor_team']))]

        if not (local_jugo_local.empty | local_jugo_visitante.empty):
            maximo_local_jugo_local = local_jugo_local[max(local_jugo_local['number_date']) == local_jugo_local['number_date']]    
            maximo_local_jugo_visitante = local_jugo_visitante[max(local_jugo_visitante['number_date']) == local_jugo_visitante['number_date']]
            maximo_local_jugo_local = maximo_local_jugo_local.set_index('number_date')
            maximo_local_jugo_visitante = maximo_local_jugo_visitante.set_index('number_date')

            
            if (maximo_local_jugo_local.index.max() > maximo_local_jugo_visitante.index.max()):
                
                stats_equipos.loc[actual,'place_local_played'] = stats_equipos.loc[actual, 'local_team']
                stats_equipos.loc[actual, 'avg_distance_place_local_played'] = 0
                stats_equipos.loc[actual,'local_dif_between_previous_game'] = stats_equipos.loc[actual, 'number_date'] - maximo_local_jugo_local.index.max()                
                stats_equipos.loc[actual,'local_dif_between_previous_game'] = (stats_equipos.loc[actual,'local_dif_between_previous_game'] / np.timedelta64(1, 'D'))
                
                if(maximo_local_jugo_local.loc[maximo_local_jugo_local.index.max(), 'local_played_local'] > -1):
                    stats_equipos.loc[actual,'local_played_local'] = maximo_local_jugo_local.loc[maximo_local_jugo_local.index.max(), 'local_played_local'] + 1
                    stats_equipos.loc[actual,'local_played_visitor'] = 0
                else:
                    stats_equipos.loc[actual,'local_played_local'] = 1
                    stats_equipos.loc[actual,'local_played_visitor'] = 0

            elif (maximo_local_jugo_local.index.max() < maximo_local_jugo_visitante.index.max()):
                
                stats_equipos.loc[actual,'place_local_played'] = maximo_local_jugo_visitante.loc[maximo_local_jugo_visitante.index.max(), 'local_team']
                stats_equipos.loc[actual,'local_dif_between_previous_game'] = stats_equipos.loc[actual, 'number_date'] - maximo_local_jugo_visitante.index.max()                
                stats_equipos.loc[actual,'local_dif_between_previous_game'] = (stats_equipos.loc[actual,'local_dif_between_previous_game'] / np.timedelta64(1, 'D'))              
                
                if(maximo_local_jugo_visitante.loc[maximo_local_jugo_visitante.index.max(), 'local_played_visitor'] > -1):
                    stats_equipos.loc[actual,'local_played_visitor'] = maximo_local_jugo_visitante.loc[maximo_local_jugo_visitante.index.max(), 'local_played_visitor'] + 1
                    stats_equipos.loc[actual,'local_played_local'] = 0
                else:
                    stats_equipos.loc[actual,'local_played_visitor'] = 1
                    stats_equipos.loc[actual,'local_played_local'] = 0
                    
        elif not (local_jugo_local.empty):
                        
            maximo_local_jugo_local = local_jugo_local[max(local_jugo_local['number_date']) == local_jugo_local['number_date']]
            maximo_local_jugo_local = maximo_local_jugo_local.set_index('number_date')
            
            stats_equipos.loc[actual,'place_local_played'] = stats_equipos.loc[actual, 'local_team']
            stats_equipos.loc[actual, 'avg_distance_place_local_played'] = 0
            stats_equipos.loc[actual,'local_dif_between_previous_game'] = stats_equipos.loc[actual, 'number_date'] - maximo_local_jugo_local.index.max()
            stats_equipos.loc[actual,'local_dif_between_previous_game'] = (stats_equipos.loc[actual,'local_dif_between_previous_game'] / np.timedelta64(1, 'D'))         
            
            if(maximo_local_jugo_local.loc[maximo_local_jugo_local.index.max(), 'local_played_local'] > -1):
                stats_equipos.loc[actual,'local_played_local'] = maximo_local_jugo_local.loc[maximo_local_jugo_local.index.max(), 'local_played_local'] + 1
                stats_equipos.loc[actual,'local_played_visitor'] = 0
            else:
                stats_equipos.loc[actual,'local_played_local'] = 1
                stats_equipos.loc[actual,'local_played_visitor'] = 0
        
        elif not (local_jugo_visitante.empty):

            maximo_local_jugo_visitante = local_jugo_visitante[max(local_jugo_visitante['number_date']) == local_jugo_visitante['number_date']]
            maximo_local_jugo_visitante = maximo_local_jugo_visitante.set_index('number_date')
            
            stats_equipos.loc[actual,'place_local_played'] = maximo_local_jugo_visitante.loc[maximo_local_jugo_visitante.index.max(), 'local_team']
            stats_equipos.loc[actual,'local_dif_between_previous_game'] = stats_equipos.loc[actual, 'number_date'] - maximo_local_jugo_visitante.index.max()        
            stats_equipos.loc[actual,'local_dif_between_previous_game'] = (stats_equipos.loc[actual,'local_dif_between_previous_game'] / np.timedelta64(1, 'D'))
            
            if(maximo_local_jugo_visitante.loc[maximo_local_jugo_visitante.index.max(), 'local_played_visitor'] > -1):
                stats_equipos.loc[actual,'local_played_visitor'] = maximo_local_jugo_visitante.loc[maximo_local_jugo_visitante.index.max(), 'local_played_visitor'] + 1
                stats_equipos.loc[actual,'local_played_local'] = 0
            else:
                stats_equipos.loc[actual,'local_played_visitor'] = 1
                stats_equipos.loc[actual,'local_played_local'] = 0            
        
        if not (visitante_jugo_local.empty | visitante_jugo_visitante.empty):
                
            maximo_visitante_jugo_local = visitante_jugo_local[max(visitante_jugo_local['number_date']) == visitante_jugo_local['number_date']]
            maximo_visitante_jugo_visitante = visitante_jugo_visitante[max(visitante_jugo_visitante['number_date']) == visitante_jugo_visitante['number_date']]
            maximo_visitante_jugo_local = maximo_visitante_jugo_local.set_index('number_date')
            maximo_visitante_jugo_visitante = maximo_visitante_jugo_visitante.set_index('number_date')

            if (maximo_visitante_jugo_local.index.max() > maximo_visitante_jugo_visitante.index.max()):
                
                stats_equipos.loc[actual,'place_visitor_played'] = maximo_visitante_jugo_local.loc[maximo_visitante_jugo_local.index.max(),'local_team']
                stats_equipos.loc[actual,'visitor_dif_between_previous_game'] = stats_equipos.loc[actual,'number_date'] - maximo_visitante_jugo_local.index.max()                
                stats_equipos.loc[actual,'visitor_dif_between_previous_game'] = (stats_equipos.loc[actual,'visitor_dif_between_previous_game'] / np.timedelta64(1, 'D'))     
                
                if(maximo_visitante_jugo_local.loc[maximo_visitante_jugo_local.index.max(), 'visitor_played_local'] > -1):
                    stats_equipos.loc[actual,'visitor_played_local'] = maximo_visitante_jugo_local.loc[maximo_visitante_jugo_local.index.max(),'visitor_played_local'] + 1
                    stats_equipos.loc[actual,'visitor_played_visitor'] = 0
                else:
                    stats_equipos.loc[actual,'visitor_played_local'] = 1
                    stats_equipos.loc[actual,'visitor_played_visitor'] = 0

            elif (maximo_visitante_jugo_local.index.max() < maximo_visitante_jugo_visitante.index.max()):
                
                stats_equipos.loc[actual,'place_visitor_played'] = maximo_visitante_jugo_visitante.loc[maximo_visitante_jugo_visitante.index.max(), 'local_team']
                stats_equipos.loc[actual,'visitor_dif_between_previous_game'] = stats_equipos.loc[actual, 'number_date'] - maximo_visitante_jugo_visitante.index.max()                
                stats_equipos.loc[actual,'visitor_dif_between_previous_game'] = (stats_equipos.loc[actual,'visitor_dif_between_previous_game'] / np.timedelta64(1, 'D'))      
                
                if(maximo_visitante_jugo_visitante.loc[maximo_visitante_jugo_visitante.index.max(), 'visitor_played_visitor'] > -1):
                    stats_equipos.loc[actual,'visitor_played_visitor'] = maximo_visitante_jugo_visitante.loc[maximo_visitante_jugo_visitante.index.max(), 'visitor_played_visitor'] + 1
                    stats_equipos.loc[actual,'visitor_played_local'] = 0
                else:
                    stats_equipos.loc[actual,'visitor_played_visitor'] = 1
                    stats_equipos.loc[actual,'visitor_played_local'] = 0
                    
        elif not (visitante_jugo_local.empty):            
            
            maximo_visitante_jugo_local = visitante_jugo_local[max(visitante_jugo_local['number_date']) == visitante_jugo_local['number_date']]
            maximo_visitante_jugo_local = maximo_visitante_jugo_local.set_index('number_date')
            
            stats_equipos.loc[actual,'place_visitor_played'] = maximo_visitante_jugo_local.loc[maximo_visitante_jugo_local.index.max(), 'local_team']
            stats_equipos.loc[actual,'visitor_dif_between_previous_game'] = stats_equipos.loc[actual, 'number_date'] - maximo_visitante_jugo_local.index.max()                
            stats_equipos.loc[actual,'visitor_dif_between_previous_game'] = (stats_equipos.loc[actual,'visitor_dif_between_previous_game'] / np.timedelta64(1, 'D'))    
            
            if(maximo_visitante_jugo_local.loc[maximo_visitante_jugo_local.index.max(), 'visitor_played_local'] > -1):
                    
                    stats_equipos.loc[actual,'visitor_played_local'] = maximo_visitante_jugo_local.loc[maximo_visitante_jugo_local.index.max(), 'visitor_played_local'] + 1
                    stats_equipos.loc[actual,'visitor_played_visitor'] = 0
            else:
                    stats_equipos.loc[actual,'visitor_played_local'] = 1
                    stats_equipos.loc[actual,'visitor_played_visitor'] = 0
                    
        elif not (visitante_jugo_visitante.empty):
            
            maximo_visitante_jugo_visitante = visitante_jugo_visitante[max(visitante_jugo_visitante['number_date']) == visitante_jugo_visitante['number_date']]
            maximo_visitante_jugo_visitante = maximo_visitante_jugo_visitante.set_index('number_date')
            
            stats_equipos.loc[actual,'place_visitor_played'] = maximo_visitante_jugo_visitante.loc[maximo_visitante_jugo_visitante.index.max(), 'local_team']
            stats_equipos.loc[actual,'visitor_dif_between_previous_game'] = stats_equipos.loc[actual, 'number_date'] - maximo_visitante_jugo_visitante.index.max()                
            stats_equipos.loc[actual,'visitor_dif_between_previous_game'] = (stats_equipos.loc[actual,'visitor_dif_between_previous_game'] / np.timedelta64(1, 'D'))
            
            if(maximo_visitante_jugo_visitante.loc[maximo_visitante_jugo_visitante.index.max(), 'visitor_played_visitor'] > -1):
                stats_equipos.loc[actual,'visitor_played_visitor'] = maximo_visitante_jugo_visitante.loc[maximo_visitante_jugo_visitante.index.max(), 'visitor_played_visitor'] + 1
                stats_equipos.loc[actual,'visitor_played_local'] = 0
            else:
                stats_equipos.loc[actual,'visitor_played_visitor'] = 1
                stats_equipos.loc[actual,'visitor_played_local'] = 0

C:\Users\34680\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\34680\Anaconda3\lib\site-packages\ipykernel_launcher.py:37: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\34680\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\34680\Anaconda3\lib\site-packages\ipykernel_launcher.py:40: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\34680\Anaconda3\lib\site-packages\ipykernel_launcher.py:42: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\34680\Anaconda3\lib\site-packages\ipykernel_launcher.py:43: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [14]:
distancias.rename(columns={'Equipo A':'local_team','Equipo B':'visitor_team'}, inplace=True)
stats_equipos = pd.merge(stats_equipos, distancias, on=['local_team', 'visitor_team'], how='left')
stats_equipos.rename(columns={'Distancia':'distance_between_stadiums'}, inplace=True)

In [15]:
distancias.rename(columns={'visitor_team':'place_local_played'}, inplace=True)
stats_equipos = pd.merge(stats_equipos, distancias, on=['local_team', 'place_local_played'], how='left')
stats_equipos.rename(columns={'Distancia':'distance_local_traveled'}, inplace=True)

In [16]:
distancias.rename(columns={'place_local_played':'place_visitor_played'}, inplace=True)
stats_equipos = pd.merge(stats_equipos, distancias, on=['local_team', 'place_visitor_played'], how='left')
stats_equipos.rename(columns={'Distancia':'distance_visitor_traveled'}, inplace=True)

In [17]:
for actual in stats_equipos.index:
    if (stats_equipos.loc[actual, 'avg_distance_place_local_played'] == 0):
        stats_equipos.loc[actual, 'distance_local_traveled'] = 0
    if (stats_equipos.loc[actual, 'distance_local_traveled'] != 'NaN'):
        stats_equipos.loc[actual, 'avg_distance_place_local_played'] = stats_equipos.loc[actual, 'distance_local_traveled'] / stats_equipos.loc[actual,'local_dif_between_previous_game']
    if (stats_equipos.loc[actual, 'distance_visitor_traveled'] != 'NaN'):
        stats_equipos.loc[actual, 'avg_distance_place_visitor_played'] = stats_equipos.loc[actual, 'distance_visitor_traveled'] / stats_equipos.loc[actual,'visitor_dif_between_previous_game']


In [18]:
stats_equipos = stats_equipos.drop(columns =["previous_10_days", 'previous_date','two_days_ago', 'three_days_ago', 'visitor_prorrogue', 'local_prorrogue']) 

In [19]:
pd.options.display.max_rows = 30
pd.options.display.max_columns = 300

In [20]:
stats_equipos.to_csv('./input_extendido.csv')

In [21]:
stats_equipos.head()

,Date,Year,Season,local_team,visitor_team,Points,Opponent_Points,Result,LOCAL_Racha,ID Partido,LOCAL_Ultimos10Victorias,LOCAL_Ultimos10Derrotas,VISITANTE_Ultimos10Victorias,VISITANTE_Ultimos10Derrotas,VISITANTE_Racha,local_Conf_position,local_Win,local_Lose,local_Percentagewl,local_Dif_leader,local_Home_win,local_Home_lose,local_Away_win,local_Away_lose,local_Div_win,local_Div_lose,local_Cnf_win,local_Cnf_lose,local_Icf_win,local_Icf_lose,visitor_Conf_position,visitor_Win,visitor_Lose,visitor_Percentagewl,visitor_Dif_leader,visitor_Home_win,visitor_Home_lose,visitor_Away_win,visitor_Away_lose,visitor_Div_win,visitor_Div_lose,visitor_Cnf_win,visitor_Cnf_lose,visitor_Icf_win,visitor_Icf_lose,local_fg,local_fga,local_fg3,local_fg3a,local_ft,local_fta,local_orb,local_drb,local_trb,local_ast,local_stl,local_blk,local_tov,local_pf,local_pts,local_ft_pct,local_fg_pct,local_fg3_pct,visitor_fg,visitor_fga,visitor_fg3,visitor_fg3a,visitor_ft,visitor_fta,visitor_orb,visitor_drb,visitor_trb,visitor_ast,visitor_stl,visitor_blk,visitor_tov,visitor_pf,visitor_pts,visitor_ft_pct,visitor_fg_pct,visitor_fg3_pct,Sueldo local,Sueldo visitante,Local_Division,Local_Conferencia,Visitor_Division,Visitor_Conferencia,LOCAL_AWS_MEDIO_AGRUPADO,VISITOR_AWS_MEDIO_AGRUPADO,LOCAL_WP_MEDIO_AGRUPADO,VISITOR_WP_MEDIO_AGRUPADO,number_date,local_dif_between_previous_game,visitor_dif_between_previous_game,local_played_previous_date,visitor_played_previous_date,local_played_two_days_ago,visitor_played_two_days_ago,local_played_three_days_ago,visitor_played_three_days_ago,local_played_prorrogue_previous_date,visitor_played_prorrogue_previous_date,local_played_prorrogue_two_days_ago,visitor_played_prorrogue_two_days_ago,local_played_prorrogue_three_days_ago,visitor_played_prorrogue_three_days_ago,local_played_local,local_played_visitor,visitor_played_visitor,visitor_played_local,place_local_played,place_visitor_played,avg_distance_place_visitor_played,avg_distance_place_local_played,distance_between_stadiums,distance_local_traveled,distance_visitor_traveled
0,Oct 30,2015,2016,Philadelphia 76ers,Utah Jazz,71,99,0,-1,/boxscores/201510300PHI.html,0.0,2.0,1.0,1.0,-1,11.0,0.0,1.0,0.0,1.5,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,24.0,0.0,1.0,0.000000,1.5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,10,29,2,4,11,16,2,15,17,8,1,2,8,13,33,68.750000,34.482759,50.000000,17,42,3,7,13,15,6,17,23,9,2,4,1,9,50,86.666667,40.476190,42.857143,44886505,60751941,Atlantic Division,Este,Northwest Division,Oeste,6.605000,3.146154,0.070539,0.046698,2015-10-30,2,2,0,0,1,1,0,0,0,0,1,1,0,0,0,1,1,0,Boston Celtics,Detroit Pistons,313.5,822.0,2964.0,1644.0,627.0
1,Nov 2,2015,2016,Philadelphia 76ers,Cleveland Cavaliers,100,107,0,-2,/boxscores/201511020PHI.html,0.0,3.0,3.0,1.0,2,10.0,0.0,2.0,0.0,2.5,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,6.0,2.0,1.0,0.666667,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,19,41,3,9,13,15,6,17,23,12,5,2,8,10,54,86.666667,46.341463,33.333333,19,43,3,13,8,12,6,16,22,11,2,3,9,9,49,66.666667,44.186047,23.076923,44886505,104254519,Atlantic Division,Este,Central Division,Este,5.163636,5.216667,0.046183,0.083295,2015-11-02,3,3,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,Philadelphia 76ers,Cleveland Cavaliers,488.0,0.0,1464.0,0.0,1464.0
2,Nov 7,2015,2016,Philadelphia 76ers,Orlando Magic,97,105,0,-5,/boxscores/201511070PHI.html,0.0,6.0,3.0,4.0,1,14.0,0.0,5.0,0.0,5.0,0.0,2.0,0.0,3.0,0.0,1.0,0.0,4.0,0.0,1.0,13.0,2.0,4.0,0.333333,3.5,1.0,2.0,1.0,2.0,0.0,1.0,1.0,2.0,1.0,2.0,19,40,4,15,2,4,5,17,22,11,2,7,9,8,44,50.000000,47.500000,26.666667,21,47,3,8,6,8,7,16,23,14,6,2,4,6,51,75.000000,44.680851,37.500000,44886505,84941368,Atlantic Division,Este,Southeast Division,Este,6.270250,5.535667,0.067151,0.068664,2015-11-07,1,1,1,1,0,0,1,1,1,1,0,0,1,1,0,1,0,1,Cleveland Cavaliers,Orlando Magic,1806.0,1464.0,1806.0,1464.0,1806.0
3,Nov 9,2015,2016,Philadelphia 76ers,Chicago Bulls,88,111,0,-6,/boxscores/201511090PHI.html,0.0,7.0,5.0,3.0,-1,14.0,0.0,6.0,0.0,6.0,0.0,3.0,0.0,3.0,0.0,1.0,0.0,5.0,0.0,1.0,5.0,4.0,3.0,0